In [1]:
from datetime import date
import pandas as pd
import requests
import time

env = 'dev'

organizations_filter = catalog.load('params:openaire_extract_options.organizations_filter')
researchproducts_filter = catalog.load('params:openaire_extract_options.researchproducts_filter')
ror_filter_value = catalog.load('params:openaire_extract_options.ror_filter_value')
access_token = catalog.load('params:openaire_extract_options.access_token')
refresh_token = catalog.load('params:openaire_extract_options.refresh_token')

print(f'organizations_filter: {organizations_filter}')
print(f'researchproducts_filter: {researchproducts_filter}')
print(f'ror_filter_value: {ror_filter_value}')
print(f'env: {env}')


                    INFO     Loading data from                                                  ]8;id=21517;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=215707;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             params:openaire_extract_options.organizations_filter                                  
                             (MemoryDataset)...                                                                    

                    INFO     Loading data from                                                  ]8;id=254853;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=414802;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             params:openaire_extract_options.researchproducts_filter                               
                             (MemoryDataset)...                                                                    

                    INFO     Loading data from params:openaire_extract_options.ror_filter_value ]8;id=777380;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=487489;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (MemoryDataset)...                                                                    

                    INFO     Loading data from params:openaire_extract_options.access_token     ]8;id=586316;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=842040;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (MemoryDataset)...                                                                    

                    INFO     Loading data from params:openaire_extract_options.refresh_token    ]8;id=491487;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=662604;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (MemoryDataset)...                                                                    

organizations_filter: pid
researchproducts_filter: relOrganizationId
ror_filter_value: https://ror.org/01tjs6929
env: dev


In [2]:
def refresh_access_token(refresh_token):
    """Get a new access_token using the refresh_token."""
    refresh_url = f"https://services.openaire.eu/uoa-user-management/api/users/getAccessToken?refreshToken={refresh_token}"
    response = requests.get(refresh_url)
    if response.status_code == 200:
        return response.json().get("access_token")
    else:
        raise Exception(f"Failed to refresh token: {response.status_code}")


In [3]:
def get_with_refresh(url, headers, params, refresh_token, max_refresh_attempts=3, context=""):
    attempts = 0
    current_access_token = headers.get("Authorization", "").replace("Bearer ", "")

    while True:
        response = requests.get(url, headers=headers, params=params)

        if response.status_code != 403:
            return response, current_access_token

        if attempts >= max_refresh_attempts:
            raise Exception("Máximo de intentos para refrescar el token alcanzado. Abortando.")

        print(f"Access token expired or invalid {context}. Refreshing token...")
        new_token = refresh_access_token(refresh_token)
        if not new_token:
            raise Exception("No se pudo refrescar el access token.")

        current_access_token = new_token
        headers["Authorization"] = f"Bearer {current_access_token}"
        attempts += 1


In [4]:
def openaire_extract_researchproduct(filter_param, ror_filter_value, access_token, refresh_token, env):
    cursor = '*'
    organizations_base_url = 'https://api.openaire.eu/graph/v1/organizations'
    research_base_url = 'https://api.openaire.eu/graph/v2/researchProducts'
    iteration_limit = 2
    iteration_count = 0
    page_size = 50
    max_retries = 5
    retry_wait = 5
    max_refresh_attempts = 3

    request_headers = {
        "accept": "application/json",
        "Authorization": f"Bearer {access_token}"
    }

    org_query_params = {
        "pid": ror_filter_value,
    }

    # Resolve OpenAIRE organization id from ROR
    while True:
        response, access_token = get_with_refresh(
            organizations_base_url,
            request_headers,
            org_query_params,
            refresh_token,
            max_refresh_attempts=max_refresh_attempts,
            context="while fetching organization",
        )

        if response.status_code != 200:
            raise Exception(f"Failed to fetch organization for ROR {ror_filter_value}: {response.status_code}")

        data = response.json()
        results = data.get("results", [])
        if not results:
            raise Exception(f"No organization found for ROR {ror_filter_value}")

        organization_id = results[0].get('id')
        if not organization_id:
            raise Exception("No se encontró id de organización en la respuesta.")

        print(f"OpenAIRE organization id resolved from ROR {ror_filter_value}: {organization_id}")
        break

    query_params = {
        filter_param: organization_id,
        "pageSize": page_size,
        "cursor": cursor
    }

    while True:
        response, access_token = get_with_refresh(
            research_base_url,
            request_headers,
            query_params,
            refresh_token,
            max_refresh_attempts=max_refresh_attempts,
            context="while fetching research products",
        )

        if response.status_code != 200:
            raise Exception(f"Failed to retrieve data: {response.status_code}")

        api_response = response.json()
        print(f"Iteration count: {iteration_count}")
        print(f"GET {response.url}")

        # Build DataFrame with the first page of results
        df = pd.DataFrame.from_dict(api_response["results"])

        # Update cursor
        cursor = api_response["header"].get("nextCursor", None)
        query_params["cursor"] = cursor

        # Iterate pagination via cursor
        while cursor:
            if env == "dev" and iteration_count >= iteration_limit:
                break

            iteration_count += 1
            print(f"Iteration count: {iteration_count}")
            print(f"GET {response.url}")
            time.sleep(2)

            # Retries when rate limited (429)
            retries = 0
            while retries < max_retries:
                response, access_token = get_with_refresh(
                    research_base_url,
                    request_headers,
                    query_params,
                    refresh_token,
                    max_refresh_attempts=max_refresh_attempts,
                    context="during execution",
                )

                if response.status_code == 429:
                    retries += 1
                    print(f"Rate limit hit. Retry {retries}/{max_retries}. Waiting {retry_wait} seconds...")
                    time.sleep(retry_wait)
                    retry_wait *= 2  # Exponential backoff
                else:
                    break

            if response.status_code != 200:
                print(f"Failed to retrieve data at iteration {iteration_count}: {response.status_code}")
                break

            api_response = response.json()

            if not api_response.get("results"):
                print("No more results. Stopping iteration.")
                break

            df_tmp = pd.DataFrame.from_dict(api_response["results"])
            df = pd.concat([df, df_tmp], ignore_index=True)

            cursor = api_response["header"].get("nextCursor", None)
            query_params["cursor"] = cursor

        df[filter_param] = organization_id

        df['load_datetime'] = date.today()

        return df, df.head(1000)


In [5]:
df, df_dev = openaire_extract_researchproduct(
    researchproducts_filter,
    ror_filter_value,
    access_token,
    refresh_token,
    env,
)


Access token expired or invalid while fetching organization. Refreshing token...
OpenAIRE organization id resolved from ROR https://ror.org/01tjs6929: openorgs____::40b9f835648a3e0d057d6917dd7e54d5
Iteration count: 0
GET https://api.openaire.eu/graph/v2/researchProducts?relOrganizationId=openorgs____%3A%3A40b9f835648a3e0d057d6917dd7e54d5&pageSize=50&cursor=%2A
Iteration count: 1
GET https://api.openaire.eu/graph/v2/researchProducts?relOrganizationId=openorgs____%3A%3A40b9f835648a3e0d057d6917dd7e54d5&pageSize=50&cursor=%2A
Iteration count: 2
GET https://api.openaire.eu/graph/v2/researchProducts?relOrganizationId=openorgs____%3A%3A40b9f835648a3e0d057d6917dd7e54d5&pageSize=50&cursor=AoI%2FD2E5YWM1MGY1NzZhYTo6NmFmNjA2NTlhMzU3NGNjZTg2MTg1M2VmN2YyOTlhODYIP4AAAA%3D%3D


In [6]:
df.head(2)

,authors,openAccessColor,publiclyFunded,type,language,countries,subjects,mainTitle,subTitle,descriptions,...,indicators,projects,organizations,communities,collectedFrom,instances,isGreen,isInDiamondJournal,relOrganizationId,load_datetime
0,"[{'fullName': 'Susevich, María Laura', 'name':...",gold,False,publication,"{'code': 'spa', 'label': 'Spanish; Castilian'}",None,"[{'subject': {'scheme': 'keyword', 'value': 'p...",Dicistrovirus from the pollinator community fo...,None,[Los Dicistrovirus son una familia de virus qu...,...,"{'citationImpact': {'citationCount': 0.0, 'inf...",None,[{'legalName': 'Universidad Nacional de La Pam...,"[{'code': 'knowmad', 'label': 'Knowmad Institu...",[{'key': 'openaire____::0b74b6a356bbf23c245f9a...,"[{'alternateIdentifiers': [{'scheme': 'doi', '...",False,False,openorgs____::40b9f835648a3e0d057d6917dd7e54d5,2025-11-21
1,"[{'fullName': 'Steffen, Kevin Denis', 'name': ...",gold,False,publication,"{'code': 'spa', 'label': 'Spanish; Castilian'}",None,"[{'subject': {'scheme': 'keyword', 'value': 'g...",Variability in the growth rates of Saanen kids...,None,[This study was carried out to determine the v...,...,"{'citationImpact': {'citationCount': 0.0, 'inf...",None,[{'legalName': 'National Scientific and Techni...,None,[{'key': 'openaire____::0b74b6a356bbf23c245f9a...,"[{'alternateIdentifiers': [{'scheme': 'doi', '...",False,False,openorgs____::40b9f835648a3e0d057d6917dd7e54d5,2025-11-21
